In [1]:
import numpy as np

In [2]:
# Hardcoded 10x10 similarity matrix for the example (P1 to P10)
def get_example_similarity_matrix():
    return np.array([
        [1.0, 0.9, 0.85, 0.6, 0.3, 0.2, 0.4, 0.25, 0.3, 0.35],
        [0.9, 1.0, 0.8, 0.55, 0.25, 0.15, 0.35, 0.2, 0.25, 0.3],
        [0.85, 0.8, 1.0, 0.65, 0.35, 0.25, 0.45, 0.3, 0.35, 0.4],
        [0.6, 0.55, 0.65, 1.0, 0.75, 0.7, 0.8, 0.65, 0.6, 0.55],
        [0.3, 0.25, 0.35, 0.75, 1.0, 0.85, 0.9, 0.8, 0.75, 0.7],
        [0.2, 0.15, 0.25, 0.7, 0.85, 1.0, 0.8, 0.9, 0.85, 0.8],
        [0.4, 0.35, 0.45, 0.8, 0.9, 0.8, 1.0, 0.85, 0.8, 0.75],
        [0.25, 0.2, 0.3, 0.65, 0.8, 0.9, 0.85, 1.0, 0.9, 0.85],
        [0.3, 0.25, 0.35, 0.6, 0.75, 0.85, 0.8, 0.9, 1.0, 0.9],
        [0.35, 0.3, 0.4, 0.55, 0.7, 0.8, 0.75, 0.85, 0.9, 1.0]
    ])


In [3]:

def select_genes(data, num_genes_to_select=1000):
    variances = np.var(data, axis=0)
    top_gene_indices = np.argsort(variances)[::-1][:num_genes_to_select]
    return data[:, top_gene_indices]


def compute_similarity_matrix(data):
    m = data.shape[0]
    similarity_matrix = np.zeros((m, m))
    for i in range(m):
        for j in range(i + 1, m):
            dist = np.sqrt(np.sum((data[i] - data[j]) ** 2))
            similarity_matrix[i, j] = dist
            similarity_matrix[j, i] = dist
    max_dist = np.max(similarity_matrix)
    if max_dist > 0:
        similarity_matrix = similarity_matrix / max_dist
    similarity_matrix = 1 - similarity_matrix
    np.fill_diagonal(similarity_matrix, 1)
    return similarity_matrix

# Step 3: Find neighbors and compute density (number of neighbors)
def find_neighbors(similarity_matrix, r):
    m = similarity_matrix.shape[0]
    neighbor_counts = np.zeros(m)
    neighbors_list = [[] for _ in range(m)]
    for i in range(m):
        # Find neighbors with similarity > r (exclude self)
        neighbors = np.where(similarity_matrix[i] > r)[0]
        neighbors = neighbors[neighbors != i]
        neighbor_counts[i] = len(neighbors)
        neighbors_list[i] = neighbors.tolist()
    return neighbor_counts, neighbors_list

# Step 4 & 5: Form clusters starting with highest density sample
def form_clusters(similarity_matrix, r):
    m = similarity_matrix.shape[0]
    neighbor_counts, neighbors_list = find_neighbors(similarity_matrix, r)


    print("\nStep 3: Neighbor and Density Calculation")
    print("--------------------------------------")
    for i in range(m):
        print(f"Patient P{i+1}: Neighbors = {[p+1 for p in neighbors_list[i]]}, Density = {int(neighbor_counts[i])}")

    clusters = []
    assigned = np.zeros(m, dtype=bool)
    cluster_num = 1

    print("\nSteps 4 & 5: Forming Clusters")
    print("-----------------------------")
    while not all(assigned):
        # Find unassigned sample with highest density
        unassigned_indices = np.where(~assigned)[0]
        if len(unassigned_indices) == 0:
            break
        neighbor_counts_unassigned = neighbor_counts[unassigned_indices]
        max_density_idx = unassigned_indices[np.argmax(neighbor_counts_unassigned)]
        density = neighbor_counts[max_density_idx]

        # Mark lead patient as assigned
        assigned[max_density_idx] = True

        # Form cluster with this sample and its unassigned neighbors
        neighbors = np.where(similarity_matrix[max_density_idx] > r)[0]
        neighbors = neighbors[neighbors != max_density_idx]  # Exclude self
        cluster = [max_density_idx]
        for neighbor in neighbors:
            if not assigned[neighbor]:
                cluster.append(neighbor)
                assigned[neighbor] = True


        print(f"Forming Cluster {cluster_num}:")
        print(f"  Lead Patient: P{int(max_density_idx)+1} (Density = {int(density)})")
        print(f"  Neighbors: {[int(p)+1 for p in neighbors]}")
        print(f"  Cluster Patients: {[int(p)+1 for p in cluster]}")

        clusters.append(cluster)
        cluster_num += 1

    return clusters

# Step 6: Merge clusters with >50-safe% similar neighbors
def merge_clusters(clusters, similarity_matrix, r):
    print("\nStep 6: Merging Clusters")
    print("------------------------")
    merged = []
    used = np.zeros(len(clusters), dtype=bool)
    final_cluster_num = 1

    for i in range(len(clusters)):
        if used[i]:
            continue
        current_cluster = clusters[i]
        merged.append(current_cluster)
        used[i] = True
        current_cluster_id = final_cluster_num
        print(f"\nChecking Cluster {current_cluster_id}: {[int(p)+1 for p in current_cluster]}")

        for j in range(i + 1, len(clusters)):
            if used[j]:
                continue
            other_cluster = clusters[j]
            print(f"  Comparing with Cluster {j+1}: {[int(p)+1 for p in other_cluster]}")

            # Find neighbors of each cluster
            neighbors_i = set()
            neighbors_j = set()
            for sample in current_cluster:
                neighbors_i.update(np.where(similarity_matrix[sample] > r)[0])
            for sample in other_cluster:
                neighbors_j.update(np.where(similarity_matrix[sample] > r)[0])

            # Check if common neighbors > 50% of smaller cluster
            common_neighbors = neighbors_i.intersection(neighbors_j)
            smaller_size = min(len(current_cluster), len(other_cluster))
            threshold = 0.5 * smaller_size
            print(f"    Common Neighbors: {[p+1 for p in common_neighbors]}")
            print(f"    Smaller Cluster Size: {smaller_size}, Threshold: {threshold}")
            print(f"    Number of Common Neighbors: {len(common_neighbors)}")

            if len(common_neighbors) > threshold:
                print(f"    Merging Cluster {j+1} into Cluster {current_cluster_id}")
                merged[-1].extend(other_cluster)
                used[j] = True
            else:
                print(f"    No merge")

        final_cluster_num += 1

    return merged


def cluster_patients(r=0.8):

    similarity_matrix = get_example_similarity_matrix()
    print("Step 2: Similarity Matrix")
    print("------------------------")
    print(similarity_matrix)


    clusters = form_clusters(similarity_matrix, r)
    print(f"\nInitial Clusters Formed: {len(clusters)}")


    final_clusters = merge_clusters(clusters, similarity_matrix, r)
    print(f"\nFinal Clusters After Merging: {len(final_clusters)}")

    print("\nFinal Clusters")
    print("--------------")
    for i, cluster in enumerate(final_clusters):
        print(f"Cluster {i + 1}: {len(cluster)} patients, IDs: {[int(p) + 1 for p in cluster]}")

    return final_clusters


In [4]:
clusters = cluster_patients(r=0.8)

Step 2: Similarity Matrix
------------------------
[[1.   0.9  0.85 0.6  0.3  0.2  0.4  0.25 0.3  0.35]
 [0.9  1.   0.8  0.55 0.25 0.15 0.35 0.2  0.25 0.3 ]
 [0.85 0.8  1.   0.65 0.35 0.25 0.45 0.3  0.35 0.4 ]
 [0.6  0.55 0.65 1.   0.75 0.7  0.8  0.65 0.6  0.55]
 [0.3  0.25 0.35 0.75 1.   0.85 0.9  0.8  0.75 0.7 ]
 [0.2  0.15 0.25 0.7  0.85 1.   0.8  0.9  0.85 0.8 ]
 [0.4  0.35 0.45 0.8  0.9  0.8  1.   0.85 0.8  0.75]
 [0.25 0.2  0.3  0.65 0.8  0.9  0.85 1.   0.9  0.85]
 [0.3  0.25 0.35 0.6  0.75 0.85 0.8  0.9  1.   0.9 ]
 [0.35 0.3  0.4  0.55 0.7  0.8  0.75 0.85 0.9  1.  ]]

Step 3: Neighbor and Density Calculation
--------------------------------------
Patient P1: Neighbors = [2, 3], Density = 2
Patient P2: Neighbors = [1], Density = 1
Patient P3: Neighbors = [1], Density = 1
Patient P4: Neighbors = [], Density = 0
Patient P5: Neighbors = [6, 7], Density = 2
Patient P6: Neighbors = [5, 8, 9], Density = 3
Patient P7: Neighbors = [5, 8], Density = 2
Patient P8: Neighbors = [6, 7, 9, 10